In [1]:
import dgl 
import torch as th

C:\Users\18438\AppData\Local\Continuum\anaconda3\envs\dgl\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\18438\AppData\Local\Continuum\anaconda3\envs\dgl\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\18438\AppData\Local\Continuum\anaconda3\envs\dgl\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Using backend: pytorch


稀疏矩阵

In [2]:
import scipy.sparse as sp
spmat = sp.rand(100, 100, density=0.05) 

In [3]:
spmat

<100x100 sparse matrix of type '<class 'numpy.float64'>'
	with 500 stored elements in COOrdinate format>

In [4]:
dgl.from_scipy(spmat)

Graph(num_nodes=100, num_edges=500,
      ndata_schemes={}
      edata_schemes={})

networkx

所以当将无向的NetworkX图转换为 DGLGraph 对象时，DGL会在内部将1条无向边转换为2条有向边。 使用有向的NetworkX图 networkx.DiGraph 可避免该行为

In [5]:
import networkx as nx
nx_g = nx.path_graph(5) 

dgl.from_networkx(nx_g) 

Graph(num_nodes=5, num_edges=8,
      ndata_schemes={}
      edata_schemes={})

dataframe

In [13]:
import pandas as pd
import os

In [17]:
data_path = 'WWW20-Hands-on-Tutorial-master/basic_tasks/data/'

In [18]:
os.listdir(data_path)

['edges.csv', 'gen_data.py', 'nodes.csv']

In [19]:
nodes = pd.read_csv(data_path + 'nodes.csv')
edges = pd.read_csv(data_path + 'edges.csv')

In [21]:
nodes.head()

,Id,Club,Age
0,0,Mr. Hi,45
1,1,Mr. Hi,33
2,2,Mr. Hi,36
3,3,Mr. Hi,31
4,4,Mr. Hi,41


In [22]:
edges.head()

,Src,Dst,Weight
0,0,1,0.318451
1,0,2,0.551215
2,0,3,0.227416
3,0,4,0.266919
4,0,5,0.475449


In [24]:
src = edges['Src'].to_numpy()
dst = edges['Dst'].to_numpy()

In [30]:
nodes['Id'].nunique(), edges.shape[0]

(34, 156)

In [26]:
g = dgl.graph((src, dst))

In [27]:
g

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={}
      edata_schemes={})

为图的节点添加年龄和俱乐部特征  
为图的边添加权重特征

In [37]:
age = th.tensor(nodes['Age'].to_numpy()).float() / 100
g.ndata['age'] = age 

In [38]:
nodes['Club'].unique()

array(['Mr. Hi', 'Officer'], dtype=object)

In [39]:
import torch.nn.functional as F

In [42]:
club = nodes['Club'].to_list()
club = th.tensor([c == 'Officer' for c in club]).long()

In [43]:
club

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [45]:
club_onehot = F.one_hot(club)
print(club_onehot[:3])

tensor([[1, 0],
        [1, 0],
        [1, 0]])


In [46]:
g.ndata.update({'club' : club, 'club_onehot' : club_onehot})

In [47]:
g

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'age': Scheme(shape=(), dtype=torch.float32), 'club': Scheme(shape=(), dtype=torch.int64), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})

In [48]:
# 去除某个特征
del g.ndata['club']

In [49]:
g

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'age': Scheme(shape=(), dtype=torch.float32), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={})

In [51]:
edge_weight = th.tensor(edges['Weight'].to_numpy())
g.edata['weight'] = edge_weight
print(g)

Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'age': Scheme(shape=(), dtype=torch.float32), 'club_onehot': Scheme(shape=(2,), dtype=torch.int64)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
